# Procesamiento inicial de datos – Grupo 5

Este notebook realiza el primer paso del análisis:

- Revisa si ya existen los archivos procesados (`.csv`) con los datos limpios.
- Si no existen, los genera a partir de las bases originales de la EPH (`.xlsx`).
- Guarda dos versiones:
  - **Base completa**: todos los registros de personas + hogares (sin filtrar).
  - **Base filtrada**: solo personas de la región pampeana, mayores de 18 y con ingreso declarado.

Luego muestra los primer 5 registros de cada base para entender cómo están estructuradas y validar que se hayan generado correctamente.


In [8]:
import os
import pandas as pd
import numpy as np
from pipeline_simple import construir_base_limpia

# Archivos de entrada/salida
PERSONAS = "usu_individual_T324.xlsx"
HOGARES = "usu_hogar_T324.xlsx"
CSV_COMPLETO = "base_completa_personas_hogares.csv"
CSV_FILTRADO = "base_pampeana_analizada.csv"

# Ejecutar pipeline si los archivos no existen
if os.path.exists(CSV_COMPLETO) and os.path.exists(CSV_FILTRADO):
    print("✅ CSVs encontrados. Cargando datos procesados...")
    df_completo = pd.read_csv(CSV_COMPLETO)
    df_filtrado = pd.read_csv(CSV_FILTRADO)
else:
    print("CSVs no encontrados. Ejecutando pipeline de procesamiento...")
    df_completo, df_filtrado = construir_base_limpia(PERSONAS, HOGARES,
                                                      out_csv_completo=CSV_COMPLETO,
                                                      out_csv_filtrado=CSV_FILTRADO)

✅ CSVs encontrados. Cargando datos procesados...


In [9]:
# Mostramos 5 registros aleatorios de la base completa
# Esta base incluye a todas las personas y hogares, útil para análisis generales.

print("Muestra de la base completa (sin filtrar):")
display(df_completo.sample(5))

Muestra de la base completa (sin filtrar):


,id_vivienda,id_hogar,region,sexo,fecha_nacimiento,edad,estado_actividad,nivel_educativo,ingreso,prestamo_personas,prestamo_banco,compra_cuotas
1941,TQSMNORWYHMMKOCDEGIBJ00809881,1,41,1,1963-10-01 00:00:00,61,3,4,520000.0,2,2,2
13242,TQRMNOSXYHJNPUCDEIJAH00867555,1,1,1,2024-08-06 00:00:00,-1,4,7,0.0,2,2,2
4829,TQRMNOTWQHJOKQCDEFNFF00861868,1,41,2,1995-03-18 00:00:00,29,1,4,600000.0,2,1,1
9276,TQRMNOQWYHJOKPCDEFLID00866054,1,43,2,2011-01-15 00:00:00,13,3,3,0.0,2,2,2
36348,TQRMNOPTVHJMKTCDEHNHB00865963,1,42,1,1979-11-17 00:00:00,44,1,4,480000.0,1,2,1


In [10]:
# Mostramos 5 registros aleatorios de la base filtrada
# Esta es la base que usamos para los análisis del informe: región pampeana, mayores de 18 y con ingreso.

print("Muestra de la base filtrada (región Pampeana, mayores de 18 con ingreso):")
display(df_filtrado.sample(5))

Muestra de la base filtrada (región Pampeana, mayores de 18 con ingreso):


,id_vivienda,id_hogar,region,sexo,fecha_nacimiento,edad,estado_actividad,nivel_educativo,ingreso,prestamo_personas,prestamo_banco,compra_cuotas
1662,TQRMNOSTQHJMKOCDEIOAH00865905,1,43,1,1900-01-01 00:00:00,50,1,5,70000.0,1,2,1
3766,TQRMNOQSVHJNMOCDEFKID00864189,1,43,2,1900-01-01 00:00:00,23,3,3,70000.0,2,2,1
5578,TQRMNOPPXHMLMNCDEGJBF00852838,1,43,2,1987-03-02 00:00:00,37,1,5,200000.0,2,2,1
383,TQRMNOQTRHMMKNCDEIGED00811781,1,43,1,1900-01-01 00:00:00,21,3,1,160000.0,2,2,1
205,TQRMNOPPTHMOKRCDEGJBF00810089,1,43,1,1978-01-24 00:00:00,46,1,4,550000.0,2,2,1


## Unidad 4 - Probabilidad

### Aplicacion de la probabilidad condicional

¿Cuál es la probabilidad de que ocurra un evento A, sabiendo que ya ocurrió otro evento B?

$P(A|B) = \frac {P(A∩B)}{P(B)}$

### Queremos responder:

**¿Cuál es la probabilidad de que una persona esté ocupada, sabiendo que tiene educación universitaria completa?**

Traducimos a símbolos:

A = persona ocupada

B = persona con educación universitaria completa

Entonces:

$P(Ocupado|Universitario) = \frac {P(Ocupado ∩ Universitario)}{P(Universitario)}$


- Si P(Ocupado | Universitario) > P(Ocupado) significa que tener educación superior aumenta las chances de estar ocupado.

- Si son iguales, entonces no hay diferencia (educación no parece influir en ocupación).

- Si es menor, entonces tener título no se asocia a mejor ocupación.

Esto no prueba causalidad, pero sí señala una asociación estadística importante.

### Calculamos:

- P(Ocupado): proporción general de personas ocupadas.

- P(Ocupado | Universitario): proporción de ocupados entre quienes tienen título universitario completo.

Luego comparamos y sacamos conclusiones.

In [16]:
# Creamos una serie booleana que indica si cada persona tiene educación universitaria completa
universitario = df_filtrado['nivel_educativo'] == 7

# Otra serie booleana que indica si la persona está ocupada
ocupado = df_filtrado['estado_actividad'] == 1

# Probabilidad condicional: P(ocupado | universitario)
# Contamos cuántas personas cumplen ambas condiciones (ocupado y universitario)
# y lo dividimos por el total de universitarios
p_ocupado_dado_universitario = (ocupado & universitario).sum() / universitario.sum()

# Probabilidad general: P(ocupado)
# Calculamos cuántas personas están ocupadas sobre el total de personas
p_ocupado_general = ocupado.mean()

# Mostramos los resultados redondeados
print("P(Ocupado) =", round(p_ocupado_general, 3))
print("P(Ocupado | Universitario) =", round(p_ocupado_dado_universitario, 3))

#Mostramos las cantidades
print("Cantidad de ocupados: ", ocupado.sum())
print("Cant. Universitarios:", universitario.sum())



P(Ocupado) = 0.636
P(Ocupado | Universitario) = 0.195
Cantidad de ocupados:  4477
Cant. Universitarios: 41


## Conclusión

P(Ocupado) = 63.6 %

P(Ocupado | Universitario) = 19.5 %

¿No debería ser al revés?
Esperaríamos que los universitarios tuvieran mayor tasa de ocupación, no menor. Este resultado es llamativo y contraintuitivo.

Esta estimación se basa en solo 41 casos dentro de la muestra. Dado el tamaño tan reducido, la proporción no es confiable para sacar conclusiones firmes. Sería necesario ampliar el análisis a más observaciones o agrupar niveles educativos para obtener resultados más estables.